In [ ]:
# recipient-notebooks/Module 2 - Delta Sharing Deep Dive/Includes/_common.ipynb

In [0]:
class DAConfig:
    """Configuration object with attribute access."""
    def __init__(self, config_dict):
        for key, value in config_dict.items():
            setattr(self, key, value)
    
    def __repr__(self):
        attrs = ', '.join([f"{k}={v}" for k, v in self.__dict__.items()])
        return f"DAConfig({attrs})"

def create_da_objects(config_values):
    """
    Create DA configuration object and SQL variables from config values.
    
    Args:
        config_values: List of tuples containing (key, value) pairs
    
    Returns:
        DAConfig: Configuration object with attribute access
    """
    # Create dictionary from config_values
    config_dict = {key: value for key, value in config_values}
    
    # Create DA object with attribute access for Python
    DA = DAConfig(config_dict)
    
    # Create a temp view storing the configuration as key-value pairs
    spark.createDataFrame(
        [(key, value) for key, value in config_dict.items()],
        ["key", "value"]
    ).createOrReplaceTempView("da_config_temp")
    
    # Create SQL dictionary var (map) from the temp view
    spark.sql("""
        CREATE OR REPLACE TEMP VIEW da_map_view AS
        SELECT map_from_arrays(collect_list(key), collect_list(value)) as config_map
        FROM da_config_temp
    """)
    
    # Declare the DA variable
    spark.sql("DECLARE OR REPLACE VARIABLE DA MAP<STRING, STRING>")
    
    # Set the DA variable from the temp view
    spark.sql("SET VAR DA = (SELECT config_map FROM da_map_view)")
    
    # Clean up temp views
    spark.sql("DROP VIEW IF EXISTS da_config_temp")
    spark.sql("DROP VIEW IF EXISTS da_map_view")
    
    return DA

In [0]:
def display_config_values(config_values):
    """
    Displays list of key-value pairs as rows of HTML text and textboxes
    :param config_values: list of (key, value) tuples

    Returns
    ----------
    HTML output displaying the config values

    Example
    --------
    DA.display_config_values([('catalog',DA.catalog_name),('schema',DA.schema_name)])
    """
    html = """<table style="width:100%">"""
    for name, value in config_values:
        html += f"""
        <tr>
            <td style="white-space:nowrap; width:1em">{name}:</td>
            <td><input type="text" value="{value}" style="width: 100%"></td></tr>"""
    html += "</table>"
    displayHTML(html)